In [1]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pyttsx3
import threading
import queue
from flask import Flask, request, render_template, jsonify
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import os
from PIL import Image
import googlemaps
from math import radians, sin, cos, sqrt, atan2
from dotenv import load_dotenv  # For environment variables

# Load environment variables
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Load and compile the model
MODEL_PATH = 'model.h5'
try:
    model = load_model(MODEL_PATH)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# Initialize Text-to-Speech queue
queue_voice = queue.Queue()

# Ensure 'uploads' directory exists
UPLOAD_FOLDER = os.path.join(os.getcwd(), 'uploads')
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Preprocessing functions
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def equalize(img):
    return cv2.equalizeHist(img)

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

# Class labels
def getClassName(classNo):
    classes = [
        'Speed Limit 20 km/h', 'Speed Limit 30 km/h', 'Speed Limit 50 km/h', 'Speed Limit 60 km/h',
        'Speed Limit 70 km/h', 'Speed Limit 80 km/h', 'End of Speed Limit 80 km/h', 'Speed Limit 100 km/h',
        'Speed Limit 120 km/h', 'No passing', 'No passing for vehicles over 3.5 metric tons',
        'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop', 'No vehicles',
        'Vehicles over 3.5 metric tons prohibited', 'No entry', 'General caution', 'Dangerous curve to the left',
        'Dangerous curve to the right', 'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right',
        'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow',
        'Wild animals crossing', 'End of all speed and passing limits', 'Turn right ahead', 'Turn left ahead',
        'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left',
        'Roundabout mandatory', 'End of no passing', 'End of no passing by vehicles over 3.5 metric tons'
    ]
    return classes[classNo] if 0 <= classNo < len(classes) else "Unknown"

# Voice output function (runs in a separate thread)
def speak_from_queue():
    while True:
        text = queue_voice.get()  # Block until text is available in queue
        if text == "stop":  # Stop signal to terminate thread gracefully
            break
        # Initialize the engine for each prediction to avoid blocking
        local_engine = pyttsx3.init()
        local_engine.setProperty('rate', 150)  # Adjust speaking speed
        voices = local_engine.getProperty('voices')
        local_engine.setProperty('voice', voices[1].id)  # Set to female voice
        local_engine.say(text)
        local_engine.runAndWait()
        local_engine.stop()

# Start voice thread
voice_thread = threading.Thread(target=speak_from_queue, daemon=True)
voice_thread.start()

# Prediction function
def model_predict(img_path, model):
    try:
        img = Image.open(img_path)
        img = img.resize((32, 32))
        img = np.asarray(img)
        img = preprocessing(img)
        img = img.reshape(1, 32, 32, 1)

        predictions = model.predict(img)
        classIndex = np.argmax(predictions, axis=1)[0]
        preds = getClassName(classIndex)

        # Add the prediction to the voice queue for processing
        queue_voice.put(f"{preds}")

        return preds
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Error during prediction"

# Home route
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/help')
def help():
    return render_template('help.html')

# Prediction route with JSON response
@app.route('/predict', methods=['POST'])
def upload():
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file part'})

        f = request.files['file']
        if f.filename == '':
            return jsonify({'error': 'No selected file'})

        if f:
            filename = secure_filename(f.filename)
            file_path = os.path.join(UPLOAD_FOLDER, filename)
            f.save(file_path)

            preds = model_predict(file_path, model)
            return jsonify({'prediction': preds})

        return jsonify({'error': 'Error'})
    except Exception as e:
        print(f"Error in file upload: {e}")
        return jsonify({'error': 'Internal server error'})

# Initialize Google Maps client
GMAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')  # Use environment variable
gmaps = googlemaps.Client(key=GMAPS_API_KEY)

# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    a = sin(dLat/2) * sin(dLat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dLon/2) * sin(dLon/2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Hospitals route
@app.route('/hospitals', methods=['GET', 'POST'])
def hospitals():
    if request.method == 'POST':
        lat = request.form.get('latitude')
        lng = request.form.get('longitude')
        radius_km = request.form.get('radius', '5')  # Default to 5 km if empty

        # Validate inputs
        try:
            lat = float(lat)
            lng = float(lng)
            radius_km = float(radius_km)
            if not (1 <= radius_km <= 50):
                raise ValueError("Radius must be between 1 and 50 km.")
        except ValueError as e:
            return render_template('hospitals.html', error=str(e))

        try:
            # Convert km to meters for Google API
            radius_m = int(radius_km * 1000)
            places = gmaps.places_nearby(
                location=(lat, lng),
                radius=radius_m,
                type='hospital'
            )
            hospitals = []
            for place in places.get('results', []):
                if 'geometry' in place and 'location' in place['geometry']:
                    hospital_lat = place['geometry']['location']['lat']
                    hospital_lng = place['geometry']['location']['lng']
                    distance = calculate_distance(lat, lng, hospital_lat, hospital_lng)
                    place['distance'] = distance
                    hospitals.append(place)

            # Sort hospitals by distance (nearest first)
            hospitals = sorted(hospitals, key=lambda x: x['distance'])

            return render_template(
                'hospitals.html',
                hospitals=hospitals,
                lat=lat,
                lng=lng,
                radius_km=radius_km
            )
        except Exception as e:
            return render_template('hospitals.html', error=str(e))
    return render_template('hospitals.html')


@app.route('/restaurants', methods=['GET', 'POST'])
def restaurants():
    if request.method == 'POST':
        lat = request.form.get('latitude')
        lng = request.form.get('longitude')
        radius_km = request.form.get('radius', '5')  # Default to 5 km if empty

        # Validate inputs
        try:
            lat = float(lat)
            lng = float(lng)
            radius_km = float(radius_km)
            if not (1 <= radius_km <= 50):
                raise ValueError("Radius must be between 1 and 50 km.")
        except ValueError as e:
            return render_template('restaurants.html', error=str(e))

        try:
            # Convert km to meters for Google API
            radius_m = int(radius_km * 1000)
            places = gmaps.places_nearby(
                location=(lat, lng),
                radius=radius_m,
                type='restaurant'
            )
            restaurants = []
            for place in places.get('results', []):
                if 'geometry' in place and 'location' in place['geometry']:
                    restaurant_lat = place['geometry']['location']['lat']
                    restaurant_lng = place['geometry']['location']['lng']
                    distance = calculate_distance(lat, lng, restaurant_lat, restaurant_lng)
                    place['distance'] = distance
                    restaurants.append(place)

            # Sort restaurants by distance (nearest first)
            restaurants = sorted(restaurants, key=lambda x: x['distance'])

            return render_template(
                'restaurants.html',
                restaurants=restaurants,
                lat=lat,
                lng=lng,
                radius_km=radius_km
            )
        except Exception as e:
            return render_template('restaurants.html', error=str(e))
    return render_template('restaurants.html')


@app.route('/gas-stations', methods=['GET', 'POST'])
def gas_stations():
    if request.method == 'POST':
        lat = request.form.get('latitude')
        lng = request.form.get('longitude')
        radius_km = request.form.get('radius', '5')  # Default to 5 km if empty

        # Validate inputs
        try:
            lat = float(lat)
            lng = float(lng)
            radius_km = float(radius_km)
            if not (1 <= radius_km <= 50):
                raise ValueError("Radius must be between 1 and 50 km.")
        except ValueError as e:
            return render_template('gas_stations.html', error=str(e))

        try:
            # Convert km to meters for Google API
            radius_m = int(radius_km * 1000)
            places = gmaps.places_nearby(
                location=(lat, lng),
                radius=radius_m,
                type='gas_station'
            )
            gas_stations = []
            for place in places.get('results', []):
                if 'geometry' in place and 'location' in place['geometry']:
                    gas_station_lat = place['geometry']['location']['lat']
                    gas_station_lng = place['geometry']['location']['lng']
                    distance = calculate_distance(lat, lng, gas_station_lat, gas_station_lng)
                    place['distance'] = distance
                    gas_stations.append(place)

            # Sort gas stations by distance (nearest first)
            gas_stations = sorted(gas_stations, key=lambda x: x['distance'])

            return render_template(
                'gas_stations.html',
                gas_stations=gas_stations,
                lat=lat,
                lng=lng,
                radius_km=radius_km
            )
        except Exception as e:
            return render_template('gas_stations.html', error=str(e))
    return render_template('gas_stations.html')

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.192.13:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:12] "GET /static/css/main.css HTTP/1.1" 304 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:12] "GET /static/js/main.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:14] "GET /static/favicon.png HTTP/1.1" 304 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:16] "GET /hospitals HTTP/1.1" 200 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:16] "GET /static/css/main.css HTTP/1.1" 304 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:16] "GET /static/js/main.js HTTP/1.1" 304 -
INFO:werkzeug:192.168.192.13 - - [09/Feb/2025 13:57:24] "POST /ho